# Bayes Classifier on Iris Dataset

## Importing Dependencies

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
df=pd.read_csv('Iris.csv')
df.head()
df.sample(5)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
141,142,6.9,3.1,5.1,2.3,Iris-virginica
67,68,5.8,2.7,4.1,1.0,Iris-versicolor
5,6,5.4,3.9,1.7,0.4,Iris-setosa
85,86,6.0,3.4,4.5,1.6,Iris-versicolor
34,35,4.9,3.1,1.5,0.1,Iris-setosa


In [5]:
df.value_counts('Species')

Species
Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: count, dtype: int64

## Assign codes to different classes

In [6]:
df['class'] = np.where(df['Species'] == 'Iris-setosa', 1, np.where(df['Species'] == 'Iris-versicolor', 2, 3))
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,class
0,1,5.1,3.5,1.4,0.2,Iris-setosa,1
1,2,4.9,3.0,1.4,0.2,Iris-setosa,1
2,3,4.7,3.2,1.3,0.2,Iris-setosa,1
3,4,4.6,3.1,1.5,0.2,Iris-setosa,1
4,5,5.0,3.6,1.4,0.2,Iris-setosa,1


## Multivariate Normal Distribution Function

In [7]:
def mvnorm(x,mean,cov):
    # no. of predictors
    p=X.shape[1]
    #Mahalanobis Distance-D2
    inv_cov = np.linalg.inv(cov)
    D2 = (x - mean)@ inv_cov @ (x - mean).T
    fx = 1 / (((2 * np.pi)**(p / 2)) * (np.linalg.det(cov)**0.5))*np.exp(-0.5*D2)
    return fx

## Separate Predictors and Response

In [8]:
X=df.drop(columns=['Id','Species','class'])
Y=df['class']
X.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


## Calculate Distribution Parameters-- Mean Vectors and Covariance Matrix

In [12]:
#Calculate Covariance Matrix
cov1=X[Y==1].cov().to_numpy()
cov2=X[Y==2].cov().to_numpy()
cov3=X[Y==3].cov().to_numpy()
cov3

array([[0.40434286, 0.09376327, 0.3032898 , 0.04909388],
       [0.09376327, 0.10400408, 0.07137959, 0.04762857],
       [0.3032898 , 0.07137959, 0.30458776, 0.04882449],
       [0.04909388, 0.04762857, 0.04882449, 0.07543265]])

In [13]:
#Calculating class wise Mean Vectors
mean_vectors = df.groupby('class')[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']].mean() #Class wise mean vectors
mean_vectors

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
class,,,,
1,5.006,3.418,1.464,0.244
2,5.936,2.770,4.260,1.326
3,6.588,2.974,5.552,2.026


In [14]:
mv_1 = mean_vectors.loc[1].to_numpy()  # Mean vector for class 1
mv_2 = mean_vectors.loc[2].to_numpy()  # Mean vector for class 2
mv_3 = mean_vectors.loc[3].to_numpy()  # Mean vector for class 3
mv_1 #view mean vector for class 1

array([5.006, 3.418, 1.464, 0.244])

In [15]:
x=X.iloc[0].to_numpy()

In [16]:
#Checking invocation of MVNorm function
mvnorm(x,mv_1,cov1)

13.725594445123013

## Calculating apriori probabilities for Bayesian Classifier

In [17]:
p1=df[df['class']==1].shape[0]/df.shape[0]
p2=df[df['class']==2].shape[0]/df.shape[0]
p3=df[df['class']==3].shape[0]/df.shape[0]

## Using entire dataset for training

In [19]:
vars=X.to_numpy()
y=np.array([])
for var in vars:
    prob1=mvnorm(var,mv_1,cov1)*p1
    prob2=mvnorm(var,mv_2,cov2)*p2
    prob3=mvnorm(var,mv_3,cov3)*p3
    if prob1==max(prob1,prob2,prob3):
        y=np.append(y,1)
    elif prob2==max(prob1,prob2,prob3):
        y=np.append(y,2)
    else:
        y=np.append(y,3)
y

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 3., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.])

### Calculating Training Accuracy

In [22]:
accuracy = np.mean(y == Y.to_numpy())
accuracy*100

98.0

## Splitting Train test data

### 80-20 Split

In [23]:
#80% train, 20%test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=42)

#### Calculating parameters for train data

In [24]:
#Calculating class wise Mean Vectors for 80% data
mv_1 = np.mean(X_train[Y_train==1],axis=0).to_numpy() # Mean vector for class 1
mv_2 = np.mean(X_train[Y_train==2],axis=0).to_numpy() # Mean vector for class 2
mv_3 = np.mean(X_train[Y_train==3],axis=0).to_numpy() # Mean vector for class 3
mv_1 #view mean vector for class 1

array([4.99  , 3.44  , 1.4525, 0.2425])

In [30]:
#Calculate Class wise Covariance Matrix
cov1=X_train[Y_train==1].cov().to_numpy()
cov2=X_train[Y_train==2].cov().to_numpy()
cov3=X_train[Y_train==3].cov().to_numpy()
cov3

array([[0.42754386, 0.10114035, 0.31429825, 0.04947368],
       [0.10114035, 0.10175439, 0.09280702, 0.06026316],
       [0.31429825, 0.09280702, 0.29325236, 0.05210526],
       [0.04947368, 0.06026316, 0.05210526, 0.08421053]])

#### Calculating prior probabilities

In [26]:
p1=(Y_train==1).sum()/Y_train.shape[0]
p2=(Y_train==2).sum()/Y_train.shape[0]
p3=(Y_train==3).sum()/Y_train.shape[0]
p3

0.325

#### Calculating training accuracy

In [27]:
vars=X_train.to_numpy()
y_train=np.array([])
for var in vars:
    prob1=mvnorm(var,mv_1,cov1)*p1
    prob2=mvnorm(var,mv_2,cov2)*p2
    prob3=mvnorm(var,mv_3,cov3)*p3
    if prob1==max(prob1,prob2,prob3):
        y_train=np.append(y_train,1)
    elif prob2==max(prob1,prob2,prob3):
        y_train=np.append(y_train,2)
    else:
        y_train=np.append(y_train,3)
y_train
accuracy=np.mean(y_train==Y_train)
accuracy*100

97.5

#### Using the 20% test data

In [31]:
vars=X_test.to_numpy()
y_pred=np.array([])
for var in vars:
    prob1=mvnorm(var,mv_1,cov1)*p1
    prob2=mvnorm(var,mv_2,cov2)*p2
    prob3=mvnorm(var,mv_3,cov3)*p3
    if prob1==max(prob1,prob2,prob3):
        y_pred=np.append(y_pred,1)
    elif prob2==max(prob1,prob2,prob3):
        y_pred=np.append(y_pred,2)
    else:
        y_pred=np.append(y_pred,3)
y_pred

array([2., 1., 3., 2., 2., 1., 2., 3., 3., 2., 3., 1., 1., 1., 1., 2., 3.,
       2., 2., 3., 1., 3., 1., 3., 3., 3., 3., 3., 1., 1.])

#### Calculating test accuracy

In [32]:
accuracy=np.mean(y_pred==Y_test)
accuracy*100

96.66666666666667

### 70-30 Split

In [33]:
#70% train, 30%test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

#### Calculating parameters for train data

In [34]:
#Calculating class wise Mean Vectors for 80% data
mv_1 = np.mean(X_train[Y_train==1],axis=0).to_numpy() # Mean vector for class 1
mv_2 = np.mean(X_train[Y_train==2],axis=0).to_numpy() # Mean vector for class 2
mv_3 = np.mean(X_train[Y_train==3],axis=0).to_numpy() # Mean vector for class 3
mv_1 #view mean vector for class 1

array([4.96451613, 3.36129032, 1.46774194, 0.24516129])

In [35]:
#Calculate Class wise Covariance Matrix
cov1=X_train[Y_train==1].cov().to_numpy()
cov2=X_train[Y_train==2].cov().to_numpy()
cov3=X_train[Y_train==3].cov().to_numpy()
cov3

array([[0.43414414, 0.09777027, 0.31913664, 0.04939189],
       [0.09777027, 0.09897898, 0.08758258, 0.06146396],
       [0.31913664, 0.08758258, 0.29644144, 0.05224474],
       [0.04939189, 0.06146396, 0.05224474, 0.0883033 ]])

#### Calculating prior probabilities

In [36]:
p1=(Y_train==1).sum()/Y_train.shape[0]
p2=(Y_train==2).sum()/Y_train.shape[0]
p3=(Y_train==3).sum()/Y_train.shape[0]
p3

0.3523809523809524

#### Calculating training accuracy

In [37]:
vars=X_train.to_numpy()
y_train=np.array([])
for var in vars:
    prob1=mvnorm(var,mv_1,cov1)*p1
    prob2=mvnorm(var,mv_2,cov2)*p2
    prob3=mvnorm(var,mv_3,cov3)*p3
    if prob1==max(prob1,prob2,prob3):
        y_train=np.append(y_train,1)
    elif prob2==max(prob1,prob2,prob3):
        y_train=np.append(y_train,2)
    else:
        y_train=np.append(y_train,3)
y_train
accuracy=np.mean(y_train==Y_train)
accuracy*100

98.09523809523809

#### Using the 30% test data

In [38]:
vars=X_test.to_numpy()
y_pred=np.array([])
for var in vars:
    prob1=mvnorm(var,mv_1,cov1)*p1
    prob2=mvnorm(var,mv_2,cov2)*p2
    prob3=mvnorm(var,mv_3,cov3)*p3
    if prob1==max(prob1,prob2,prob3):
        y_pred=np.append(y_pred,1)
    elif prob2==max(prob1,prob2,prob3):
        y_pred=np.append(y_pred,2)
    else:
        y_pred=np.append(y_pred,3)
y_pred

array([2., 1., 3., 2., 2., 1., 2., 3., 2., 2., 3., 1., 1., 1., 1., 2., 3.,
       2., 2., 3., 1., 3., 1., 3., 3., 3., 3., 3., 1., 1., 1., 1., 2., 1.,
       1., 3., 2., 1., 1., 1., 3., 2., 2., 1., 1.])

#### Test Accuracy

In [40]:
accuracy=np.mean(y_pred==Y_test)
accuracy*100

100.0

### 60-40 Split

In [41]:
#60% train, 40%test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=42)

#### Calculating parameters for train data

In [42]:
#Calculating class wise Mean Vectors for 60% data
mv_1 = np.mean(X_train[Y_train==1],axis=0).to_numpy() # Mean vector for class 1
mv_2 = np.mean(X_train[Y_train==2],axis=0).to_numpy() # Mean vector for class 2
mv_3 = np.mean(X_train[Y_train==3],axis=0).to_numpy() # Mean vector for class 3
mv_1 #view mean vector for class 1

array([4.94444444, 3.33333333, 1.46296296, 0.24074074])

In [43]:
#Calculate Covariance Matrix
#Calculate Class wise Covariance Matrix
cov1=X_train[Y_train==1].cov().to_numpy()
cov2=X_train[Y_train==2].cov().to_numpy()
cov3=X_train[Y_train==3].cov().to_numpy()
cov3

array([[0.40523185, 0.10540323, 0.27185484, 0.03918347],
       [0.10540323, 0.1       , 0.09387097, 0.05927419],
       [0.27185484, 0.09387097, 0.24387097, 0.04153226],
       [0.03918347, 0.05927419, 0.04153226, 0.08474798]])

#### Calculating prior probabilities

In [44]:
p1=(Y_train==1).sum()/Y_train.shape[0]
p2=(Y_train==2).sum()/Y_train.shape[0]
p3=(Y_train==3).sum()/Y_train.shape[0]
p3

0.35555555555555557

#### Calculating training accuracy

In [45]:
vars=X_train.to_numpy()
y_train=np.array([])
for var in vars:
    prob1=mvnorm(var,mv_1,cov1)*p1
    prob2=mvnorm(var,mv_2,cov2)*p2
    prob3=mvnorm(var,mv_3,cov3)*p3
    if prob1==max(prob1,prob2,prob3):
        y_train=np.append(y_train,1)
    elif prob2==max(prob1,prob2,prob3):
        y_train=np.append(y_train,2)
    else:
        y_train=np.append(y_train,3)
y_train
accuracy=np.mean(y_train==Y_train)
accuracy*100

98.88888888888889

#### Using the 40% test data

In [46]:
vars=X_test.to_numpy()
y_pred=np.array([])
for var in vars:
    prob1=mvnorm(var,mv_1,cov1)*p1
    prob2=mvnorm(var,mv_2,cov2)*p2
    prob3=mvnorm(var,mv_3,cov3)*p3
    if prob1==max(prob1,prob2,prob3):
        y_pred=np.append(y_pred,1)
    elif prob2==max(prob1,prob2,prob3):
        y_pred=np.append(y_pred,2)
    else:
        y_pred=np.append(y_pred,3)
y_pred

array([2., 1., 3., 2., 2., 1., 2., 3., 2., 2., 3., 1., 1., 1., 1., 2., 3.,
       2., 2., 3., 1., 3., 1., 3., 3., 3., 3., 3., 1., 1., 1., 1., 2., 1.,
       1., 3., 2., 1., 1., 1., 3., 2., 2., 1., 1., 2., 2., 3., 2., 3., 2.,
       3., 2., 1., 3., 2., 1., 1., 1., 2.])

#### Calculating test accuracy

In [47]:
accuracy=np.mean(y_pred==Y_test)
accuracy*100

98.33333333333333

### 50-50 Split

In [48]:
#50% train, 50%test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, random_state=42)

#### Calculating parameters for train data

In [49]:
#Calculating class wise Mean Vectors for 50% data
mv_1 = np.mean(X_train[Y_train==1],axis=0).to_numpy() # Mean vector for class 1
mv_2 = np.mean(X_train[Y_train==2],axis=0).to_numpy() # Mean vector for class 2
mv_3 = np.mean(X_train[Y_train==3],axis=0).to_numpy() # Mean vector for class 3
mv_1 #view mean vector for class 1

array([4.92380952, 3.30952381, 1.42857143, 0.23809524])

In [50]:
#Calculate Covariance Matrix
#Calculate Class wise Covariance Matrix
cov1=X_train[Y_train==1].cov().to_numpy()
cov2=X_train[Y_train==2].cov().to_numpy()
cov3=X_train[Y_train==3].cov().to_numpy()
cov3

array([[0.37310541, 0.06381766, 0.22485755, 0.02880342],
       [0.06381766, 0.07849003, 0.05475783, 0.06119658],
       [0.22485755, 0.05475783, 0.18763533, 0.0282906 ],
       [0.02880342, 0.06119658, 0.0282906 , 0.09564103]])

#### Calculating prior probabilities

In [51]:
p1=(Y_train==1).sum()/Y_train.shape[0]
p2=(Y_train==2).sum()/Y_train.shape[0]
p3=(Y_train==3).sum()/Y_train.shape[0]
p3

0.36

#### Using the 50% test data

In [52]:
vars=X_test.to_numpy()
y_pred=np.array([])
for var in vars:
    prob1=mvnorm(var,mv_1,cov1)*p1
    prob2=mvnorm(var,mv_2,cov2)*p2
    prob3=mvnorm(var,mv_3,cov3)*p3
    if prob1==max(prob1,prob2,prob3):
        y_pred=np.append(y_pred,1)
    elif prob2==max(prob1,prob2,prob3):
        y_pred=np.append(y_pred,2)
    else:
        y_pred=np.append(y_pred,3)
y_pred

array([2., 1., 3., 2., 2., 1., 2., 3., 2., 2., 3., 1., 1., 1., 1., 2., 3.,
       2., 2., 3., 1., 3., 1., 3., 2., 3., 3., 3., 1., 1., 1., 1., 2., 1.,
       1., 3., 2., 1., 1., 1., 3., 2., 2., 1., 1., 2., 2., 3., 2., 3., 2.,
       3., 2., 1., 3., 2., 1., 1., 1., 2., 3., 1., 1., 1., 2., 1., 2., 3.,
       1., 2., 3., 1., 3., 3., 2.])

#### Calculating test accuracy

In [53]:
accuracy=np.mean(y_pred==Y_test)
accuracy*100

97.33333333333334